下载需要的库

In [ ]:
!pip install --upgrade pip
!pip install sentencepiece
!pip install datasets
!pip install transformers

导入需要的库

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import classification_report

定义模型参数

In [ ]:
LR = 1e-5
EPOCHS = 10
BATCH_SIZE = 32
MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment" # use this to finetune the language model
#MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment" # use this to finetune the sentiment classifier


添加数据集

In [ ]:
# 将文本转换为向量
#训练集
myDataset={}
myDataset["train"]={}
myDataset["env"]={}
myDataset["train"]["data"]=[]
myDataset["train"]["label"]=[]
myDataset["env"]["data"]=[]
myDataset["env"]["label"]=[]
with open("/content/drive/MyDrive/data/train.txt","r",encoding="utf-8") as file:
    for line in file:
        label,data=line.split("|")
        myDataset["train"]["data"].append(data)
        myDataset["train"]["label"].append(int(label))
with open("/content/drive/MyDrive/data/output.txt","r",encoding="utf-8") as file:
    for line in file:
      data,label=line.split("||")
      myDataset["train"]["data"].append(data)
      myDataset["train"]["label"].append(int(label))
#验证集
with open("/content/drive/MyDrive/data/dev_en.txt","r", encoding="utf-8") as file:
    for line in file:
        label,data=line.split("|")
        myDataset["env"]["data"].append(data)
        myDataset["env"]["label"].append(int(label))
with open("/content/drive/MyDrive/data/dev_pt.txt","r", encoding="utf-8") as file:
    for line in file:
        label,data=line.split("|")
        myDataset["env"]["data"].append(data)
        myDataset["env"]["label"].append(int(label))
myDataset["train"]["data"].extend(myDataset["env"]["data"])
myDataset["train"]["label"].extend(myDataset["env"]["label"])

In [ ]:
print(len(myDataset["train"]["label"]))

18200


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_encodings = tokenizer(myDataset['train']['data'], truncation=True, padding=True)
val_encodings = tokenizer(myDataset['env']['data'], truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# 构建数据集
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
    def __len__(self):
        return len(self.label)
    def __getitem__(self,idx):
        item={key :torch.tensor(value[idx]) for key,value in self.data.items()}
        item["label"]=self.label[idx]
        return item
train_dataset = MyDataset(train_encodings, myDataset['train']['label'])
val_dataset = MyDataset(val_encodings, myDataset['env']['label'])
for val in val_dataset:
  print(val)

In [ ]:
!pip install accelerate --upgrade
!pip install transformers[torch]
training_args=TrainingArguments(
    output_dir="./result",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=100,                         # number of warmup steps for learning rate scheduler
    weight_decay=0.01,
    logging_dir='./logs',                     # directory for storing logs
    logging_steps=10,
)
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/bm", num_labels=2)
print(model)

In [ ]:
trainer = Trainer(
    model=model,                              # the instantiated 🤗 Transformers model to be trained
    args=training_args,                       # training arguments, defined above
    train_dataset=train_dataset,              # training dataset
    eval_dataset=val_dataset                  # evaluation dataset
)

trainer.train()

In [ ]:
print(model)
trainer.save_model("./results/best_model") # save best model
model.save_pretrained('./bm')

In [ ]:
test_preds_raw, test_labels , _ = trainer.predict(val_dataset)
test_preds = np.argmax(test_preds_raw, axis=-1)
print(classification_report(test_labels, test_preds, digits=3))

              precision    recall  f1-score   support

           0      0.909     0.966     0.936       145
           1      0.891     0.745     0.812        55

    accuracy                          0.905       200
   macro avg      0.900     0.855     0.874       200
weighted avg      0.904     0.905     0.902       200



In [ ]:
from sklearn.metrics import classification_report

# 假设test_preds是模型的预测结果，true_labels是真实的标签
report = classification_report(test_labels, test_preds, digits=3)

# 解析分类报告并获取每个类别的准确率和召回率
lines = report.split('\n')
precision = []
recall = []
for line in lines[2:4]:
    row = line.split()
    precision.append(float(row[1]))
    recall.append(float(row[2]))

# 输出每个类别的准确率和召回率
for i in range(2):
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
res=((1+0.49)*precision[0]*recall[0]/(0.49*precision[0]+recall[0])+(1+0.49)*precision[1]*recall[1]/(0.49*precision[1]+recall[1]))/2
print(res)

Precision: 0.909
Recall: 0.966
Precision: 0.891
Recall: 0.745
0.8820209377785959


In [ ]:
!cp -r bm /content/drive/MyDrive

In [ ]:
!cp -r tokenizer /content/drive/MyDrive